# Chat with Memory: Having Real Conversations! 🧠

One of the coolest things about chatting with AI is that it can remember what you talked about earlier in the conversation. This is called "chat history" or "context".

For example, if you ask about the weather and then ask "Why is that?", the AI will know you're asking about the weather!

Let's try having a conversation where the AI remembers what we talked about:

### Import Libraries

In [1]:
import subprocess
import os
import socket
import time
import psutil
from ollama import chat


## Starting the Ollama Server 🖥️

Before we can use any Ollama models, we need to make sure the Ollama server is running. Let's create a helper function that will:
1. Check if an Ollama server is already running
2. Start a new server only if needed
3. Keep track of the server process

This way, we won't accidentally start multiple servers!

In [3]:
def is_port_in_use(port: int = 11434) -> bool:
    """Check if the Ollama port is already in use"""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) == 0

def is_ollama_running() -> bool:
    """Check if Ollama process is running"""
    for proc in psutil.process_iter(['name']):
        try:
            if proc.info['name'] == 'ollama':
                return True
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass
    return False

def ensure_ollama_server():
    """
    Ensures that an Ollama server is running.
    Returns the process object if a new server was started, None if server was already running.
    """
    # First check if the server is already running
    if is_port_in_use() or is_ollama_running():
        print("✅ Ollama server is already running!")
        return None
    
    try:
        # Start the Ollama server
        print("🚀 Starting Ollama server...")
        proc = subprocess.Popen(
            ["ollama", "serve"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            start_new_session=True
        )
        
        # Wait for the server to start (max 10 seconds)
        for _ in range(10):
            if is_port_in_use():
                print(f"✅ Ollama server started successfully (PID: {proc.pid})")
                return proc
            time.sleep(1)
            
        raise TimeoutError("Server didn't start within 10 seconds")
        
    except Exception as e:
        print(f"❌ Error starting Ollama server: {e}")
        if 'proc' in locals():
            proc.terminate()
        raise

In [4]:
# Start the server if it's not already running
ollama_process = ensure_ollama_server()

✅ Ollama server is already running!


## Chat with Memory: Having Real Conversations! 🧠

One of the coolest things about chatting with AI is that it can remember what you talked about earlier in the conversation. This is called "chat history" or "context".

For example, if you ask about the weather and then ask "Why is that?", the AI will know you're asking about the weather!

Let's try having a conversation where the AI remembers what we talked about:


In [5]:
# This is our conversation history - it's like the AI's memory!
messages = [
    # First, we ask about the sky
    {
        'role': 'user',
        'content': 'Why is the sky blue?',
    },
    # The AI's response about the sky
    {
        'role': 'assistant',
        'content': "The sky is blue because of the way the Earth's atmosphere scatters sunlight.",
    },
    # Then we ask about weather
    {
        'role': 'user',
        'content': 'What is the weather in Tokyo?',
    },
    # The AI's response about Tokyo weather
    {
        'role': 'assistant',
        'content': 'The weather in Tokyo is typically warm and humid during the summer months, with temperatures often exceeding 30°C (86°F). The city experiences a rainy season from June to September, with heavy rainfall and occasional typhoons. Winter is mild, with temperatures rarely dropping below freezing.',
    },
]


In [6]:
def display_chat_history():
    """Show the conversation so far"""
    print("\n🗨️ Chat History:")
    for msg in messages:
        if msg['role'] == 'user':
            print(f"\n👤 You: {msg['content']}")
        else:
            print(f"🤖 AI: {msg['content']}")
    print("\n" + "="*50 + "\n")

In [7]:
# Show our chat history so far
display_chat_history()


🗨️ Chat History:

👤 You: Why is the sky blue?
🤖 AI: The sky is blue because of the way the Earth's atmosphere scatters sunlight.

👤 You: What is the weather in Tokyo?
🤖 AI: The weather in Tokyo is typically warm and humid during the summer months, with temperatures often exceeding 30°C (86°F). The city experiences a rainy season from June to September, with heavy rainfall and occasional typhoons. Winter is mild, with temperatures rarely dropping below freezing.




In [8]:
# Let's have a conversation!
print("Let's chat! (Type 'quit' to end the conversation)\n")
while True:
    # Get what the user wants to say
    user_input = input('👤 You: ')
    
    # Check if the user wants to quit
    if user_input.lower() in ['quit', 'exit', 'bye']:
        print("\n👋 Thanks for chatting!")
        break
    
    try:
        # Send the message and all previous history to the AI
        response = chat(
            'qwen3:0.6b',
            messages=[*messages, {'role': 'user', 'content': user_input}],
        )
        
        # Show the AI's response
        print(f"\n🤖 AI: {response.message.content}\n")
        
        # Add both the user's message and AI's response to the history
        messages.extend([
            {'role': 'user', 'content': user_input},
            {'role': 'assistant', 'content': response.message.content},
        ])
        
    except Exception as e:
        print(f"\n❌ Oops! Something went wrong: {e}\n")


Let's chat! (Type 'quit' to end the conversation)



👤 You:  why is it like that



🤖 AI: <think>
Okay, the user is asking why the sky is blue, and they also mentioned the weather in Tokyo. Let me start by addressing the sky question first. I should explain the reason for the blue sky, which is due to atmospheric scattering. Then, for Tokyo, I need to cover the current weather and maybe mention the seasonal patterns.

Wait, the user is using "why is it like that" as a question, so maybe they're trying to get a more detailed explanation. I should make sure my answers are clear and concise. Also, check if there's any specific information needed about Tokyo's weather that the user is interested in. Maybe they want to know about the rainy season or the temperature ranges. I should structure the response to cover both topics, ensuring that each explanation is explained clearly and logically.
</think>

The sky appears blue because sunlight is scattered by the Earth's atmosphere. Specifically, water molecules and other molecules in the air refract and disperse the sunlight,

👤 You:  quit



👋 Thanks for chatting!


### How Chat History Works 🤔

When we chat with the AI, we're doing something pretty clever:

1. We keep a list of all messages (both yours and the AI's) in the `messages` list
2. Each message has:
   - A `role`: either 'user' (you) or 'assistant' (the AI)
   - `content`: what was said

3. Every time we send a new message:
   - We include ALL previous messages
   - This helps the AI understand the context
   - The AI can refer back to what was said before

This is why the AI can understand follow-up questions like:
- "Can you explain that more?"
- "Why is that?"
- "What about in winter?"

Try asking some follow-up questions in the chat above to see how the AI uses the conversation history! 🚀
